In [1]:
import pandas as pd
import numpy as np                                                 # Numerical operations
from sklearn.compose import ColumnTransformer                      # Applies different preprocessing to different columns.
from sklearn.pipeline import Pipeline                              # Streamlines preprocessing and model training.
from sklearn.impute import SimpleImputer                           # Fills in missing data.
from sklearn.preprocessing import OneHotEncoder                    # Converts categorical data into a numerical format.

# Modeling 
from sklearn.ensemble import RandomForestRegressor                 # Predicts continuous values using multiple decision trees.
from sklearn.model_selection import train_test_split               # Splits data into training and testing sets. 

# ToDo
# 1) Import the data
# 2) Check the missing data
# 3) Check all the data is numerical 
# 4) Drop the column "Price" from the DataFrame and there is a Price section no longer exisist 
# 5) Conver the data in nums 

In [21]:
# 1) Import the data
data = pd.read_csv("data/car-sales-extended-missing-data.csv")
data.head(10)

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
5,Honda,Red,42652.0,4.0,23883.0
6,Toyota,Blue,163453.0,4.0,8473.0
7,Honda,White,NaN,4.0,20306.0
8,NaN,White,130538.0,4.0,9374.0
9,Honda,Blue,51029.0,4.0,26683.0


In [27]:
# 2) Check the missing data
data.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [29]:
# 3) Check all the data is numerical
data.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

In [35]:
# Drop the column "Price" from the DataFrame and there is a Price section no longer exisist 
data.drop(columns=["Price"], inplace=True)

In [37]:
data

,Make,Colour,Odometer (KM),Doors
0,Honda,White,35431.0,4.0
1,BMW,Blue,192714.0,5.0
2,Honda,White,84714.0,4.0
3,Toyota,White,154365.0,4.0
4,Nissan,Blue,181577.0,3.0
...,...,...,...,...
995,Toyota,Black,35820.0,4.0
996,NaN,White,155144.0,3.0
997,Nissan,Blue,66604.0,4.0
998,Honda,White,215883.0,4.0


In [ ]:
# 